In [1]:
import os
import csv
import numpy as np

In [2]:
main_dir ="/Users/08volt/Desktop/StremingML/My_results"
old_drifts = ["sea_P(y)","sea_P(y_x)","sea_P(x_y)","sine_P(y)","sine_P(y_x)","sine_P(x_y)"]
new_drifts = ["jitter","disappearing-minority","appearing-minority","appearing-clusters",
             "shapeshift","minority-share","splitting-clusters","clusters-movement","borderline"]
type_drift = ["incremental","sudden","periodic"]
unbalance_lvls = [1,2,3,4]
stats_w = ["Recall_0","Recall_1", "F1_0", "F1_1", "Gmean"]
stats_tot =["time", "memory"]
algs = ["RebalanceStream","CSMOTE","improved_UOB","improved_OOB","WEOB1","WEOB2","ESOS_ELM","UOB","OOB"]

In [3]:
def import_csv(csvfilename):
    data = []
    with open(csvfilename, "r") as scraped:
        reader = csv.reader(scraped, delimiter=',')
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                row_index += 1
                columns = [str(row_index)]+ [row[i] for i in range(len(row))]
                data.append(columns)
        #print(f"{row_index} valid rows")
        
    return data

# MEAN ALL ROWS

# OLD

In [4]:
final_dict = {}

for old in old_drifts:
    path = main_dir + "/" + old[old.index("P"):]   #old = sea_P(y)
    if old not in final_dict:
        final_dict[old] = {}
        
    for ds in os.listdir(path):
        #check if ds in old (directory giusta)
        if old[:3] == ds[:3]:
            path1 = path + "/" + ds
            
            unbalance = 0
            for u in unbalance_lvls:
                if str(u) in ds:
                    unbalance = u
            assert unbalance != 0
                        
            if unbalance not in final_dict[old]:
                final_dict[old][unbalance] = {}
                
            dtype = "sudden"
            if "G" in ds:
                dtype = "incremental"
                
            if dtype not in final_dict[old][unbalance]:
                final_dict[old][unbalance][dtype] = {}

            for alg in os.listdir(path1):
                if "DS_Store" in alg:
                    continue
                path2 = path1 + "/" + alg
                class0 = []
                class1 = []
                
                print(f"{old} {unbalance} {dtype} {alg}")
                #prendo i valori di tutti i 10 esperimenti
                for result in os.listdir(path2):
                    if "err" not in result or "DS_Store" in result:
                        continue
                    start = False
                    data = import_csv(path2+"/"+ result)
                    daily0 = []
                    daily1 = []
                    for data_row in data:
                        if data_row[1] == "learning evaluation instances":
                            start = True
                            continue
                        if not start:
                            continue
                        
                        try:
                            daily0.append(float(data_row[9]))
                        except:
                            if data_row[9] == "?":
                                daily0.append(0)
                            else:
                                print("ZERO")
                                print(data_row)
                        try:
                            daily1.append(float(data_row[10]))
                        except:
                            if data_row[10] == "?":
                                daily1.append(0)
                            else:
                                print("ONE")
                                print(data_row)
                    if len(daily0)>0 and len(daily1)>0:
                        class0.append(np.mean(daily0))
                        class1.append(np.mean(daily1))
                            
                        
                    
                assert len(class0) > 0
                assert len(class1) > 0
                assert len(class1) == len(class0)
                if np.mean(class0) != np.mean(class0):
                    print(class0)
                if np.mean(class1) != np.mean(class1):
                    print(class1)
                if alg == "OSHAT":
                    alg = "CSMOTE"
                final_dict[old][unbalance][dtype][alg] = {}
                final_dict[old][unbalance][dtype][alg]["class0"] ={} 
                final_dict[old][unbalance][dtype][alg]["class1"] = {}
                final_dict[old][unbalance][dtype][alg]["class0"]["mean"] = np.mean(class0)
                final_dict[old][unbalance][dtype][alg]["class1"]["mean"] = np.mean(class1)
                final_dict[old][unbalance][dtype][alg]["class0"]["std"] =np.std(class0)
                final_dict[old][unbalance][dtype][alg]["class1"]["std"] = np.std(class1)

                print(np.mean(class0))
                print(np.mean(class1))
                    
                    
                
        

sea_P(y) 1 sudden improved_OOB
99.04759791924309
98.01035759171211
sea_P(y) 1 sudden WEOB1
98.96768761731389
97.61824995430362
sea_P(y) 1 sudden OOB
99.04677942979217
98.14362056873725
sea_P(y) 1 sudden ESOS_ELM
89.30415619559611
73.42351874845741
sea_P(y) 1 sudden improved_UOB
98.76905095904638
96.82396823481169
sea_P(y) 1 sudden WEOB2
98.76905095904638
96.82396823481169
sea_P(y) 1 sudden UOB
99.04677942979217
98.14362056873725
sea_P(y) 1 sudden OSHAT
98.74740154470427
97.13040882292873
sea_P(y) 1 sudden RebalanceStream
98.80394056083252
97.28100012232429
sea_P(y) 3 incremental improved_OOB
87.97760221279725
80.61434114561438
sea_P(y) 3 incremental WEOB1
87.86834554227079
80.53205480765956
sea_P(y) 3 incremental OOB
87.6621930976067
80.2035661865664
sea_P(y) 3 incremental ESOS_ELM
82.41503576802805
63.45563049312749
sea_P(y) 3 incremental improved_UOB
87.57550823353623
80.18066738092452
sea_P(y) 3 incremental WEOB2
87.57550823353623
80.18066738092452
sea_P(y) 3 incremental UOB
87.3487

84.74041941764753
79.27657367425162
sea_P(y_x) 4 sudden ESOS_ELM
81.21121484849324
61.83718447652819
sea_P(y_x) 4 sudden improved_UOB
84.31262601840002
77.63836475313829
sea_P(y_x) 4 sudden WEOB2
84.31262601840002
77.63836475313829
sea_P(y_x) 4 sudden UOB
83.15382120034704
77.68755868442571
sea_P(y_x) 4 sudden OSHAT
84.36629003563084
77.88115345992912
sea_P(y_x) 4 sudden RebalanceStream
84.26079458473279
77.05709044101523
sea_P(x_y) 1 sudden improved_OOB
94.81985732593597
62.91423137356967
sea_P(x_y) 1 sudden WEOB1
92.75260764368718
57.045990490020344
sea_P(x_y) 1 sudden OOB
94.03306889377752
60.46960687460472
sea_P(x_y) 1 sudden ESOS_ELM
92.71794955463612
44.69159079693999
sea_P(x_y) 1 sudden improved_UOB
88.17297709531763
46.00493719647023
sea_P(x_y) 1 sudden WEOB2
88.17297709531763
46.00493719647023
sea_P(x_y) 1 sudden UOB
87.1901863837937
44.17483354047031
sea_P(x_y) 1 sudden OSHAT
90.63654433359105
48.68540004602371
sea_P(x_y) 1 sudden RebalanceStream
96.01050147334264
43.43740314

98.45521469529503
96.47554015018085
sine_P(y) 2 incremental OSHAT
97.637939598043
96.67757574789115
sine_P(y) 2 incremental RebalanceStream
96.84111020517761
96.36236178452944
sine_P(y) 1 incremental improved_OOB
99.05934188929521
97.53286190464264
sine_P(y) 1 incremental WEOB1
99.13086073622458
97.27711553850492
sine_P(y) 1 incremental OOB
99.20822982432088
97.30083239918017
sine_P(y) 1 incremental ESOS_ELM
94.80152454378573
77.69834450430776
sine_P(y) 1 incremental improved_UOB
98.07410251993689
93.1642670991205
sine_P(y) 1 incremental WEOB2
98.07410251993689
93.1642670991205
sine_P(y) 1 incremental UOB
98.24052884185835
93.24789205830318
sine_P(y) 1 incremental OSHAT
96.6306832781323
91.93287610734868
sine_P(y) 1 incremental RebalanceStream
97.50758873132474
94.44221354477192
sine_P(y) 2 sudden improved_OOB
99.18973690812535
98.31289630495209
sine_P(y) 2 sudden WEOB1
99.18093118409533
98.15332384505972
sine_P(y) 2 sudden OOB
99.23968565668494
98.17793071193995
sine_P(y) 2 sudden ESO

99.06591440217743
98.62537595341503
sine_P(x_y) 4 sudden WEOB2
99.06591440217743
98.62537595341503
sine_P(x_y) 4 sudden UOB
98.92480903600492
98.44219016050215
sine_P(x_y) 4 sudden OSHAT
98.85466027668578
98.3217171763931
sine_P(x_y) 4 sudden RebalanceStream
98.62096377859567
97.9880061132042
sine_P(x_y) 1 sudden improved_OOB
99.60928464226131
96.43178097755526
sine_P(x_y) 1 sudden WEOB1
99.5590943046906
96.02204704905175
sine_P(x_y) 1 sudden OOB
99.59035543419486
96.27948451782393
sine_P(x_y) 1 sudden ESOS_ELM
97.47874092398634
70.662466422781
sine_P(x_y) 1 sudden improved_UOB
98.7989872854999
90.09978104941874
sine_P(x_y) 1 sudden WEOB2
98.7989872854999
90.09978104941874
sine_P(x_y) 1 sudden UOB
98.54765017771773
88.32061046648992
sine_P(x_y) 1 sudden OSHAT
97.96231598040663
83.23099619051092
sine_P(x_y) 1 sudden RebalanceStream
99.39830670484506
94.2059822199304
sine_P(x_y) 2 incremental improved_OOB
99.54967692982501
98.2372937213344
sine_P(x_y) 2 incremental WEOB1
99.4927088203165

In [5]:
old = final_dict.copy()
old

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'class0': {'mean': 99.04759791924309,
      'std': 0.054155009436708514},
     'class1': {'mean': 98.01035759171211, 'std': 0.1043766511814312}},
    'WEOB1': {'class0': {'mean': 98.96768761731389,
      'std': 0.04148973804603072},
     'class1': {'mean': 97.61824995430362, 'std': 0.0866122586459847}},
    'OOB': {'class0': {'mean': 99.04677942979217, 'std': 0.05308974725852438},
     'class1': {'mean': 98.14362056873725, 'std': 0.11938604534246629}},
    'ESOS_ELM': {'class0': {'mean': 89.30415619559611,
      'std': 0.10278132708036258},
     'class1': {'mean': 73.42351874845741, 'std': 0.5217861393667154}},
    'improved_UOB': {'class0': {'mean': 98.76905095904638,
      'std': 0.08259704514450893},
     'class1': {'mean': 96.82396823481169, 'std': 0.16160776617706157}},
    'WEOB2': {'class0': {'mean': 98.76905095904638,
      'std': 0.08259704514450893},
     'class1': {'mean': 96.82396823481169, 'std': 0.16160776617706157}},
    'UOB

# NEW

In [6]:
final_dict = old.copy()

In [7]:
for new in new_drifts:
    path = main_dir + "/" + new 
    
    if new not in final_dict:
        final_dict[new] = {}
        
    for unbalance in range(1,5):
        if unbalance not in final_dict[new]:
            final_dict[new][unbalance] = {}


        path1 = path + "/" + str(unbalance)

        for alg in os.listdir(path1):
            if "DS_Store" in alg:
                continue
            path2 = path1 + "/" + alg
            
            alg_dtypes = {}
            alg_dtypes["incremental"] = {}
            alg_dtypes["sudden"] = {}
            alg_dtypes["periodic"] = {}
            alg_dtypes["incremental"]["class0"] = []
            alg_dtypes["sudden"]["class0"] = []
            alg_dtypes["periodic"]["class0"] = []
            alg_dtypes["incremental"]["class1"] = []
            alg_dtypes["sudden"]["class1"] = []
            alg_dtypes["periodic"]["class1"] = []
            
            
            print(f"{new} {unbalance} {alg}")
            #prendo i valori di tutti i 10 esperimenti dei 3 dtype
            
            for result in os.listdir(path2):
                if "err" not in result or "DS_Store" in result:
                    continue
                daily0 = []
                daily1 = []
                data = import_csv(path2+"/"+ result)
                start = False
                
                for data_row in data:
                    if data_row[1] == "learning evaluation instances":
                        start = True
                        continue
                    if not start:
                        continue

                    try:
                        daily0.append(float(data_row[9]))
                    except:
                        if data_row[9] == "?":
                            daily0.append(0.0)
                        else:
                            print("ZERO")
                            print(data_row)
                    try:
                        daily1.append(float(data_row[10]))
                    except:
                        if data_row[10] == "?":
                            daily1.append(0.0)
                        else:
                            print("ONE")
                            print(data_row)
                        
                    
                
                try:
                    if len(daily0)>0 and len(daily1)>0:
                        if alg == "ESOS_ELM" and new == "appearing-minority":
                            print(daily0)
                            print(daily1)
                        alg_dtypes[result[:result.index("_")]]["class0"].append(np.mean(daily0))
                        alg_dtypes[result[:result.index("_")]]["class1"].append(np.mean(daily1))
                except:
                    print(f"error with {result}")
            for k,i in alg_dtypes.items():      
                if len(i["class0"]) < 10:
                    print(f"N EXP {len(i['class0'])} in 0")
                if len(i["class1"]) < 10:
                    print(f"N EXP {len(i['class1'])} in 1")
                if k not in final_dict[new][unbalance]:
                    final_dict[new][unbalance][k] = {}
                assert alg not in final_dict[new][unbalance][k]
                final_dict[new][unbalance][k][alg] = {}
                final_dict[new][unbalance][k][alg]["class0"] ={} 
                final_dict[new][unbalance][k][alg]["class1"] = {}
                final_dict[new][unbalance][k][alg]["class0"]["mean"] = np.mean(i["class0"])
                final_dict[new][unbalance][k][alg]["class1"]["mean"] = np.mean(i["class1"])
                final_dict[new][unbalance][k][alg]["class0"]["std"] =np.std(i["class0"])
                final_dict[new][unbalance][k][alg]["class1"]["std"] = np.std(i["class1"])

                print(k)
                print(np.mean(i["class1"]))
                print(np.mean(i["class0"]))

jitter 1 improved_OOB
incremental
65.8411567920077
94.28060000477905
sudden
64.01935690674054
93.97053812977563
periodic
65.8411567920077
94.28060000477906
jitter 1 WEOB1
incremental
62.802109236841055
93.07116612364264
sudden
61.16394935258536
92.86640309692598
periodic
62.80210923684105
93.07116612364266
jitter 1 CSMOTE
incremental
42.77060637417368
86.0907328276071
sudden
47.74465185542761
88.01308046367015
periodic
42.77060637417368
86.0907328276071
jitter 1 OOB
incremental
65.30754979001283
93.96103536293327
sudden
63.521420554515714
93.57871860224529
periodic
65.30754979001284
93.96103536293326
jitter 1 ESOS_ELM
incremental
34.1052838404195
84.46844793787247
sudden
55.50938218131704
93.19340712062409
periodic
33.59323904616951
84.2229182286193
jitter 1 improved_UOB
incremental
54.0143257926746
88.6469751101486
sudden
52.02400455608985
88.76318200191722
periodic
54.01432579267458
88.64697511014859
jitter 1 WEOB2
incremental
54.0143257926746
88.6469751101486
sudden
52.0240045560898

incremental
53.12799960911136
87.82181898166012
sudden
51.10609902327991
88.02689110531067
periodic
53.12799960911135
87.82181898166012
disappearing-minority 2 RebalanceStream
incremental
48.667154171749175
96.34307314643145
sudden
45.52927411064117
96.13960045879836
periodic
48.667154171749175
96.34307314643145
disappearing-minority 3 improved_OOB
incremental
65.8411567920077
94.28060000477905
sudden
64.01935690674054
93.97053812977563
periodic
65.8411567920077
94.28060000477906
disappearing-minority 3 WEOB1
incremental
62.802109236841055
93.07116612364264
sudden
61.16394935258536
92.86640309692598
periodic
62.80210923684105
93.07116612364266
disappearing-minority 3 CSMOTE
incremental
41.05504253089475
92.53990786068297
sudden
38.63563806683284
92.81597442277179
periodic
76.68147001187914
87.67874797845482
disappearing-minority 3 OOB
incremental
65.30754979001283
93.96103536293327
sudden
63.521420554515714
93.57871860224529
periodic
65.30754979001284
93.96103536293326
disappearing-min

incremental
53.12799960911136
87.82181898166012
sudden
51.10609902327991
88.02689110531067
periodic
53.12799960911135
87.82181898166012
appearing-minority 1 RebalanceStream
incremental
48.667154171749175
96.34307314643145
sudden
45.52927411064117
96.13960045879836
periodic
48.667154171749175
96.34307314643145
appearing-minority 2 improved_OOB
incremental
65.8411567920077
94.28060000477905
sudden
64.01935690674054
93.97053812977563
periodic
65.8411567920077
94.28060000477906
appearing-minority 2 WEOB1
incremental
62.802109236841055
93.07116612364264
sudden
61.16394935258536
92.86640309692598
periodic
62.80210923684105
93.07116612364266
appearing-minority 2 CSMOTE
incremental
36.58915977598559
94.19050831335787
sudden
32.63663900704978
93.31806573333246
periodic
5.937710176418852
99.39906609072537
appearing-minority 2 OOB
incremental
65.30754979001283
93.96103536293327
sudden
63.521420554515714
93.57871860224529
periodic
65.30754979001284
93.96103536293326
appearing-minority 2 ESOS_ELM
[

incremental
53.12799960911136
87.82181898166012
sudden
51.10609902327991
88.02689110531067
periodic
53.12799960911135
87.82181898166012
appearing-minority 2 RebalanceStream
incremental
48.667154171749175
96.34307314643145
sudden
45.52927411064117
96.13960045879836
periodic
48.667154171749175
96.34307314643145
appearing-minority 3 improved_OOB
incremental
65.8411567920077
94.28060000477905
sudden
64.01935690674054
93.97053812977563
periodic
65.8411567920077
94.28060000477906
appearing-minority 3 WEOB1
incremental
62.802109236841055
93.07116612364264
sudden
61.16394935258536
92.86640309692598
periodic
62.80210923684105
93.07116612364266
appearing-minority 3 CSMOTE
incremental
41.483971972258075
94.08573897558892
sudden
39.52286839414388
94.07466192786512
periodic
6.380336459744504
98.76028982993611
appearing-minority 3 OOB
incremental
65.30754979001283
93.96103536293327
sudden
63.521420554515714
93.57871860224529
periodic
65.30754979001284
93.96103536293326
appearing-minority 3 ESOS_ELM


incremental
54.0143257926746
88.6469751101486
sudden
52.02400455608985
88.76318200191722
periodic
54.01432579267458
88.64697511014859
appearing-minority 3 UOB
incremental
53.12799960911136
87.82181898166012
sudden
51.10609902327991
88.02689110531067
periodic
53.12799960911135
87.82181898166012
appearing-minority 3 RebalanceStream
incremental
48.667154171749175
96.34307314643145
sudden
45.52927411064117
96.13960045879836
periodic
48.667154171749175
96.34307314643145
appearing-minority 4 improved_OOB
incremental
65.8411567920077
94.28060000477905
sudden
64.01935690674054
93.97053812977563
periodic
65.8411567920077
94.28060000477906
appearing-minority 4 WEOB1
incremental
62.802109236841055
93.07116612364264
sudden
61.16394935258536
92.86640309692598
periodic
62.80210923684105
93.07116612364266
appearing-minority 4 CSMOTE
incremental
44.83969503291134
92.99022128784368
sudden
41.958619467806486
93.33139677028927
periodic
6.895902654652322
98.95603321035337
appearing-minority 4 OOB
incremen

incremental
54.0143257926746
88.6469751101486
sudden
52.02400455608985
88.76318200191722
periodic
54.01432579267458
88.64697511014859
appearing-minority 4 UOB
incremental
53.12799960911136
87.82181898166012
sudden
51.10609902327991
88.02689110531067
periodic
53.12799960911135
87.82181898166012
appearing-minority 4 RebalanceStream
incremental
48.667154171749175
96.34307314643145
sudden
45.52927411064117
96.13960045879836
periodic
48.667154171749175
96.34307314643145
appearing-clusters 1 improved_OOB
incremental
65.8411567920077
94.28060000477905
sudden
64.01935690674054
93.97053812977563
periodic
65.8411567920077
94.28060000477906
appearing-clusters 1 WEOB1
incremental
62.802109236841055
93.07116612364264
sudden
61.16394935258536
92.86640309692598
periodic
62.80210923684105
93.07116612364266
appearing-clusters 1 CSMOTE
incremental
65.70502492757362
94.98153743256016
sudden
66.6381136936992
95.23742120310371
periodic
75.21938346241538
96.53459870548633
appearing-clusters 1 OOB
incrementa

incremental
54.0143257926746
88.6469751101486
sudden
52.02400455608985
88.76318200191722
periodic
54.01432579267458
88.64697511014859
shapeshift 2 UOB
incremental
53.12799960911136
87.82181898166012
sudden
51.10609902327991
88.02689110531067
periodic
53.12799960911135
87.82181898166012
shapeshift 2 RebalanceStream
incremental
48.667154171749175
96.34307314643145
sudden
45.52927411064117
96.13960045879836
periodic
48.667154171749175
96.34307314643145
shapeshift 3 improved_OOB
incremental
65.8411567920077
94.28060000477905
sudden
64.01935690674054
93.97053812977563
periodic
65.8411567920077
94.28060000477906
shapeshift 3 WEOB1
incremental
62.802109236841055
93.07116612364264
sudden
61.16394935258536
92.86640309692598
periodic
62.80210923684105
93.07116612364266
shapeshift 3 CSMOTE
incremental
76.91484266792258
86.55243194323714
sudden
76.950992055699
86.64742716648662
periodic
76.37739975516206
85.74491007796937
shapeshift 3 OOB
incremental
65.30754979001283
93.96103536293327
sudden
63.5

incremental
53.12799960911136
87.82181898166012
sudden
51.10609902327991
88.02689110531067
periodic
53.12799960911135
87.82181898166012
minority-share 4 RebalanceStream
incremental
48.667154171749175
96.34307314643145
sudden
45.52927411064117
96.13960045879836
periodic
48.667154171749175
96.34307314643145
splitting-clusters 1 improved_OOB
incremental
65.8411567920077
94.28060000477905
sudden
64.01935690674054
93.97053812977563
periodic
65.8411567920077
94.28060000477906
splitting-clusters 1 WEOB1
incremental
62.802109236841055
93.07116612364264
sudden
61.16394935258536
92.86640309692598
periodic
62.80210923684105
93.07116612364266
splitting-clusters 1 CSMOTE
incremental
56.79142694082691
92.72139034783258
sudden
58.27268311278499
93.31530805539467
periodic
66.08020107480377
94.689816454681
splitting-clusters 1 OOB
incremental
65.30754979001283
93.96103536293327
sudden
63.521420554515714
93.57871860224529
periodic
65.30754979001284
93.96103536293326
splitting-clusters 1 ESOS_ELM
increme

incremental
54.0143257926746
88.6469751101486
sudden
52.02400455608985
88.76318200191722
periodic
54.01432579267458
88.64697511014859
clusters-movement 2 UOB
incremental
53.12799960911136
87.82181898166012
sudden
51.10609902327991
88.02689110531067
periodic
53.12799960911135
87.82181898166012
clusters-movement 2 RebalanceStream
incremental
48.667154171749175
96.34307314643145
sudden
45.52927411064117
96.13960045879836
periodic
48.667154171749175
96.34307314643145
clusters-movement 3 improved_OOB
incremental
65.8411567920077
94.28060000477905
sudden
64.01935690674054
93.97053812977563
periodic
65.8411567920077
94.28060000477906
clusters-movement 3 WEOB1
incremental
62.802109236841055
93.07116612364264
sudden
61.16394935258536
92.86640309692598
periodic
62.80210923684105
93.07116612364266
clusters-movement 3 CSMOTE
incremental
67.60601181195605
78.23098143527761
sudden
66.08398803615339
80.36850550623396
periodic
75.95331715962831
86.23429988486882
clusters-movement 3 OOB
incremental
65.

In [8]:
final_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'class0': {'mean': 99.04759791924309,
      'std': 0.054155009436708514},
     'class1': {'mean': 98.01035759171211, 'std': 0.1043766511814312}},
    'WEOB1': {'class0': {'mean': 98.96768761731389,
      'std': 0.04148973804603072},
     'class1': {'mean': 97.61824995430362, 'std': 0.0866122586459847}},
    'OOB': {'class0': {'mean': 99.04677942979217, 'std': 0.05308974725852438},
     'class1': {'mean': 98.14362056873725, 'std': 0.11938604534246629}},
    'ESOS_ELM': {'class0': {'mean': 89.30415619559611,
      'std': 0.10278132708036258},
     'class1': {'mean': 73.42351874845741, 'std': 0.5217861393667154}},
    'improved_UOB': {'class0': {'mean': 98.76905095904638,
      'std': 0.08259704514450893},
     'class1': {'mean': 96.82396823481169, 'std': 0.16160776617706157}},
    'WEOB2': {'class0': {'mean': 98.76905095904638,
      'std': 0.08259704514450893},
     'class1': {'mean': 96.82396823481169, 'std': 0.16160776617706157}},
    'UOB

In [9]:
# def stats_of(_dict ):
#     values_0 = []
#     values_1 = []
#     for alg, values in _dict.items():
#         if "ELM" in alg:
#             continue
#         values_0.append(values["class0"])
#         values_1.append(values["class1"])
#     return np.mean(values_0),np.mean(values_1), np.max(values_0), np.max(values_1), np.min(values_0), np.min(values_1)


In [10]:

with open('F1_withSTD.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["drift","unbalance","type","alg","F1_0", "F1_1","F1_0_std", "F1_1_std"])
    for drift, dicts in final_dict.items():
        for unbalance, dtypes in dicts.items():
            for dtype, algs in dtypes.items():
                #mean0, mean1,max0,max1,min0,min1 = stats_of(algs)
                for alg, values in algs.items():
                    
                    print([drift,unbalance,dtype,alg])
                    print(values)

                    writer.writerow([drift,unbalance,dtype,alg,values["class0"]["mean"],values["class1"]["mean"],values["class0"]["std"],values["class1"]["std"]])

['sea_P(y)', 1, 'sudden', 'improved_OOB']
{'class0': {'mean': 99.04759791924309, 'std': 0.054155009436708514}, 'class1': {'mean': 98.01035759171211, 'std': 0.1043766511814312}}
['sea_P(y)', 1, 'sudden', 'WEOB1']
{'class0': {'mean': 98.96768761731389, 'std': 0.04148973804603072}, 'class1': {'mean': 97.61824995430362, 'std': 0.0866122586459847}}
['sea_P(y)', 1, 'sudden', 'OOB']
{'class0': {'mean': 99.04677942979217, 'std': 0.05308974725852438}, 'class1': {'mean': 98.14362056873725, 'std': 0.11938604534246629}}
['sea_P(y)', 1, 'sudden', 'ESOS_ELM']
{'class0': {'mean': 89.30415619559611, 'std': 0.10278132708036258}, 'class1': {'mean': 73.42351874845741, 'std': 0.5217861393667154}}
['sea_P(y)', 1, 'sudden', 'improved_UOB']
{'class0': {'mean': 98.76905095904638, 'std': 0.08259704514450893}, 'class1': {'mean': 96.82396823481169, 'std': 0.16160776617706157}}
['sea_P(y)', 1, 'sudden', 'WEOB2']
{'class0': {'mean': 98.76905095904638, 'std': 0.08259704514450893}, 'class1': {'mean': 96.823968234811

['appearing-minority', 3, 'incremental', 'improved_UOB']
{'class0': {'mean': 88.6469751101486, 'std': 1.3980435911245992}, 'class1': {'mean': 54.0143257926746, 'std': 2.5117828428926123}}
['appearing-minority', 3, 'incremental', 'WEOB2']
{'class0': {'mean': 88.6469751101486, 'std': 1.3980435911245992}, 'class1': {'mean': 54.0143257926746, 'std': 2.5117828428926123}}
['appearing-minority', 3, 'incremental', 'UOB']
{'class0': {'mean': 87.82181898166012, 'std': 1.7432655739675962}, 'class1': {'mean': 53.12799960911136, 'std': 2.7045084701613273}}
['appearing-minority', 3, 'incremental', 'RebalanceStream']
{'class0': {'mean': 96.34307314643145, 'std': 0.31493939287977474}, 'class1': {'mean': 48.667154171749175, 'std': 7.680482318231079}}
['appearing-minority', 3, 'sudden', 'improved_OOB']
{'class0': {'mean': 93.97053812977563, 'std': 0.5842817308352496}, 'class1': {'mean': 64.01935690674054, 'std': 2.234415026236428}}
['appearing-minority', 3, 'sudden', 'WEOB1']
{'class0': {'mean': 92.8664

{'class0': {'mean': 87.82181898166012, 'std': 1.7432655739675962}, 'class1': {'mean': 53.12799960911135, 'std': 2.704508470161328}}
['minority-share', 3, 'periodic', 'RebalanceStream']
{'class0': {'mean': 96.34307314643145, 'std': 0.31493939287977474}, 'class1': {'mean': 48.667154171749175, 'std': 7.680482318231079}}
['minority-share', 4, 'incremental', 'improved_OOB']
{'class0': {'mean': 94.28060000477905, 'std': 0.5190571371958821}, 'class1': {'mean': 65.8411567920077, 'std': 2.042882263520316}}
['minority-share', 4, 'incremental', 'WEOB1']
{'class0': {'mean': 93.07116612364264, 'std': 0.6837891695923715}, 'class1': {'mean': 62.802109236841055, 'std': 2.2397802418820056}}
['minority-share', 4, 'incremental', 'CSMOTE']
{'class0': {'mean': 88.17849780905195, 'std': 0.0}, 'class1': {'mean': 82.54291182446818, 'std': 0.0}}
['minority-share', 4, 'incremental', 'OOB']
{'class0': {'mean': 93.96103536293327, 'std': 0.5577707145552693}, 'class1': {'mean': 65.30754979001283, 'std': 1.934212729

# REALS

In [15]:
real_dict = {}
real_dict["KDDCup"] = {}
real_dict["PAKDD"] = {}
real_dict["Elec"] = {}

for i in real_dict.keys():
    path = f"/Users/08volt/Desktop/StremingML/TEST/Results_union/Real/{i}"
    for alg in os.listdir(path):
        class0 = []
        if "DS_Store" in alg or "ipynb" in alg:
            continue
        class1 = []
        if alg not in real_dict[i]:
            real_dict[i][alg] = {}
        
            
        for exp in os.listdir(path + "/" + alg): 
            if "err" not in exp or "DS_Store" in exp:
                continue
                
            data = import_csv(path + "/" + alg + "/"+ exp)
            daily0 = []
            daily1 = []
            start = False

            for data_row in data:
                if data_row[1] == "learning evaluation instances":
                    start = True
                    continue
                if not start:
                    continue

                try:
                    daily0.append(float(data_row[9]))
                except:
                    if data_row[9] == "?":
                        daily0.append(0.0)
                    else:
                        print("ZERO")
                        print(data_row)
                try:
                    daily1.append(float(data_row[10]))
                except:
                    if data_row[10] == "?":
                        daily1.append(0.0)
                    else:
                        print("ONE")
                        print(data_row)
            try:
                if len(daily0)>0 and len(daily1)>0:
                    class0.append(np.mean(daily0))
                    class1.append(np.mean(daily1))
            except:
                print(f"error with {result}")
        real_dict[i][alg]["class0"] = {}
        real_dict[i][alg]["class1"] = {}
        real_dict[i][alg]["class0"]["mean"] = np.mean(class0)
        real_dict[i][alg]["class1"]["mean"] = np.mean(class1)
        real_dict[i][alg]["class0"]["std"] = np.std(class0)
        real_dict[i][alg]["class1"]["std"] = np.std(class1)
        
                
                

In [16]:
real_dict

{'KDDCup': {'improved_OOB': {'class0': {'mean': 97.92131092513542,
    'std': 1.4210854715202004e-14},
   'class1': {'mean': 91.26893835964778, 'std': 0.0}},
  'WEOB1': {'class0': {'mean': 95.63253800339828, 'std': 0.0},
   'class1': {'mean': 91.14788385158384, 'std': 0.0}},
  'OOB': {'class0': {'mean': 97.71239503010152, 'std': 1.4210854715202004e-14},
   'class1': {'mean': 92.83696092715991, 'std': 0.0}},
  'ESOS_ELM': {'class0': {'mean': 83.12720197527338, 'std': 0.0},
   'class1': {'mean': 0.0, 'std': 0.0}},
  'improved_UOB': {'class0': {'mean': 92.06415723668195,
    'std': 1.4210854715202004e-14},
   'class1': {'mean': 89.58295496510378, 'std': 0.0}},
  'WEOB2': {'class0': {'mean': 92.4518633128902, 'std': 0.0},
   'class1': {'mean': 81.49641525620021, 'std': 0.0}},
  'UOB': {'class0': {'mean': 95.05175395125022, 'std': 1.4210854715202004e-14},
   'class1': {'mean': 94.35237962976271, 'std': 0.0}},
  'OSHAT': {'class0': {'mean': 97.32720428082052, 'std': 0.0},
   'class1': {'mean

In [17]:
with open('F1XF1_real_withSTD.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["ds","alg","F1_0", "F1_1","F1_0_std", "F1_1_std"])
    for ds, dicts in real_dict.items():
        
        for alg, values in dicts.items():
            
            if alg == "OSHAT":
                alg = "CSMOTE"
            writer.writerow([ds,alg,values["class0"]["mean"],values["class1"]["mean"],values["class0"]["std"],values["class1"]["std"]])
